In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        mlp_feature_dim = 128
        self.mlp = nn.Sequential(
            nn.Linear(features_dim, mlp_feature_dim),
            nn.ELU(),
            nn.Linear(mlp_feature_dim, mlp_feature_dim),
            nn.ELU(),
            nn.Linear(mlp_feature_dim, mlp_feature_dim),
            nn.ELU(),
        )
        self.action_head = nn.Linear(mlp_feature_dim, action_space.n)
        self.value_head = nn.Linear(mlp_feature_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.mlp.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
        
        with torch.no_grad():
            features = self.encoder(obs)
            
        features = self.mlp(features)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C17 pretrained and frozen (AnnaCNN), MLP added (128)",
                      "notes": "camera noop removed from actions"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-09-27 09:45:26,222	INFO wandb.py:170 -- Already logged into W&B.
2021-09-27 09:45:26,238	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_my_env_a44fc_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=457) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=457)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=457) 2021-09-27 09:45:30,315	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=457) 2021-09-27 09:45:30,315	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=452) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=452)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=457) 2021-09-27 09:45:37,146	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=452) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=452)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=452) /root/miniconda/envs/py37/lib/python3.7/site-packages/gy

Result for PPO_my_env_a44fc_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-27_09-46-36
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 03be24429ec34374bfc1b78d8a5a014c
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.805732337633769
          entropy_coeff: 0.009999999999999998
          kl: 0.010329088540200829
          policy_loss: 0.04055147626333767
          total_loss: 0.015616235468122694
          vf_explained_var: 0.22965596616268158
          vf_loss: 0.0010562666378165079
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.1.100
  

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_a44fc_00000,RUNNING,192.168.1.100:457,141,1683.9,141000,0,0,0,995.67


Result for PPO_my_env_a44fc_00000:
  agent_timesteps_total: 142000
  custom_metrics: {}
  date: 2021-09-27_10-13-55
  done: false
  episode_len_mean: 995.67
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 142
  experiment_id: 03be24429ec34374bfc1b78d8a5a014c
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.15
          cur_lr: 5.000000000000001e-05
          entropy: 2.103549999660916
          entropy_coeff: 0.009999999999999998
          kl: 0.009268246148892798
          policy_loss: -0.028026428570350013
          total_loss: -0.04754615492290921
          vf_explained_var: -0.8501831889152527
          vf_loss: 0.00012553793704783958
    num_agent_steps_sampled: 142000
    num_agent_steps_trained: 142000
    num_steps_sampled: 142000
    num_steps_trained: 142000
  iterations_since_restore: 142
  node_ip: 192.168.1.100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_a44fc_00000,RUNNING,192.168.1.100:457,142,1695.24,142000,0,0,0,995.67


Result for PPO_my_env_a44fc_00000:
  agent_timesteps_total: 143000
  custom_metrics: {}
  date: 2021-09-27_10-14-07
  done: false
  episode_len_mean: 995.67
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 143
  experiment_id: 03be24429ec34374bfc1b78d8a5a014c
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.15
          cur_lr: 5.000000000000001e-05
          entropy: 2.1419659548335606
          entropy_coeff: 0.009999999999999998
          kl: 0.008303343883149314
          policy_loss: -0.08544610804981656
          total_loss: -0.10540715588463677
          vf_explained_var: -0.555293619632721
          vf_loss: 0.00021311044470672237
    num_agent_steps_sampled: 143000
    num_agent_steps_trained: 143000
    num_steps_sampled: 143000
    num_steps_trained: 143000
  iterations_since_restore: 143
  node_ip: 192.168.1.100
 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_a44fc_00000,RUNNING,192.168.1.100:457,143,1706.54,143000,0,0,0,995.67
